In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import KFold, GroupKFold
from scipy.stats import pearsonr as p
import matplotlib.pyplot as plt
import seaborn as sns 

#data from : https://www.kaggle.com/columbia2131/speed-up-reading-csv-to-pickle/notebook
#train = pd.read_pickle('../input/ump-train-picklefile/train.pkl')

In [ ]:
#all_data=train.sample(50000)


In [ ]:
#features = ['investment_id']
#features+=(f'f_{i}' for i in range(300))
#y=train['target']
#train1=train[features]

In [ ]:
train = pd.read_pickle('../input/ump-train-picklefile/train.pkl')[-1500000:]


In [ ]:
kfold = GroupKFold(n_splits=5)
feats = [f for f in train.columns if f not in ['time_id', 'row_id', 'target']]
target = 'target'

In [ ]:
fold_scores = []
models = []
from catboost import CatBoostRegressor
for fold, (trn_idx, val_idx) in enumerate(kfold.split(train[feats], train[target], groups=train['time_id'])):
    xtrain, ytrain = train.iloc[trn_idx][feats], train.iloc[trn_idx][target]
    xval, yval = train.iloc[val_idx][feats], train.iloc[val_idx][target]
    model = CatBoostRegressor(learning_rate=0.1)

        
    model.fit(xtrain,ytrain,use_best_model=True,verbose=50)

    val_preds = model.predict(xval)
    
    score = p(val_preds, yval)[0]
    print(f"Fold {fold + 1}: {score}")
    fold_scores.append(score)
    models.append(model)

In [ ]:
print(f"Overall score: {np.mean(fold_scores)}")

In [ ]:
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()  

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df[feats]
    
    pred_0 = models[0].predict(test_df)
    pred_1 = models[1].predict(test_df)
    pred_2 = models[2].predict(test_df)
    pred_3 = models[3].predict(test_df)
    pred_4 = models[4].predict(test_df)
    pred = (pred_0 + pred_1 + pred_2 + pred_3 + pred_4) / 5
    sample_prediction_df['target'] = pred
    env.predict(sample_prediction_df) 

In [ ]:
#f=['f_74', 'f_135', 'f_145', 'f_108', 'f_62', 'f_241', 'f_225', 'f_163',
 #      'f_183', 'f_22', 'f_8', 'f_142', 'f_200', 'f_231', 'f_29', 'f_269']

In [ ]:
"""    from catboost import CatBoostRegressor
    from sklearn.model_selection import KFold
    model = CatBoostRegressor(learning_rate=0.1)
    # Create test array to store predictions
    # Create a KFold object
    kfold = KFold(n_splits = 5, random_state = 2021, shuffle = True)
    a=[]
    # Iterate through each fold
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train1)):
        print(f'Training fold {fold + 1}')
        x_train, x_val = train1.iloc[trn_ind], train1.iloc[val_ind]
        y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]

  
        
        model.fit(x_train,y_train,
                     use_best_model=True,
                     verbose=50)
        # Add predictions to the out of folds array
        y_pred = model.predict(x_val)
        
        a.append(pd.DataFrame({"y_train":y_val, "oof_train": y_pred}).corr()["y_train"]["oof_train"])
        
        print(pd.DataFrame({"y_train":y_val, "oof_train": y_pred}).corr()["y_train"]["oof_train"])
"""

In [ ]:
#np.sum(a)/5

In [ ]:
#0.11462616050283278# مع كاونت
#0.11106358891952903#عادي
#0.10981638462254177#0.1


In [ ]:
#print((0.11573579303847466+0.10280696695353297+0.12729788364557285+0.08767780811948664+0.11556347135564167)/5)#0.1


In [ ]:
"""
for (test_df, sample_prediction_df) in iter_test:
    test_df=test_df[features]

    preds = model.predict(test_df.values)
    
    sample_prediction_df['target'] = preds  # make your predictions here
    env.predict(sample_prediction_df)  """